In [1]:
import spacy
import en_core_web_lg
nlp = en_core_web_lg.load()

In [2]:
from nlp_utils import *
import nltk
nltk.download('wordnet')
nltk.download("stopwords")

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Jattioui\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Jattioui\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
lst_stopwords=['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren',
               "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by',
               'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don',
               "don't", 'down', 'dtype', 'during', 'each', 'few', 'for', 'from', 'further', 'ha', 'had', 'hadn', "hadn't", 
               'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', 'her', 'here', 'hers', 'herself', 'him', 
               'himself', 'his', 'how', 'i', 'if', 'in', 'into', 'is', 'isn', "isn't", 'it', "it's", 'its', 'itself', 'just', 
               'length', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 
               'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'object', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 
               'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'say', 'shan', "shan't", 'she', "she's",
               'should', "should've", 'shouldn', "shouldn't", 'so', 'some', 'such', 't', 'than', 'that', "that'll", 'the',
               'their', 'theirs', 'them', 'themselves', 'then', 'there', 'these', 'they', 'this', 'those', 'through', 'to',
               'too', 'u', 'under', 'until', 'up', 've', 'very', 'wa', 'was', 'wasn', "wasn't", 'we', 'were', 'weren',
               "weren't", 'what', 'when', 'where', 'which', 'while', 'who', 'whom', 'why', 'will', 'with', 'won', "won't",
               'wouldn', "wouldn't", 'y', 'you', "you'd", "you'll", "you're", "you've", 'your', 'yours', 'yourself',
               'yourselves']

import re
def utils_preprocess_text(text, flg_stemm=False, flg_lemm=True):
    ## clean (convert to lowercase and remove punctuations and characters and then strip)
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
            
    ## Tokenize (convert from string to list)
    lst_text = text.split()

    ## remove Stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in 
                    lst_stopwords]
                
    ## Stemming (remove -ing, -ly, ...)
    if flg_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_text = [ps.stem(word) for word in lst_text]
                
    ## Lemmatisation (convert the word into root word)
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_text = [lem.lemmatize(word) for word in lst_text]
            
    ## back to string from list
    text = " ".join(lst_text)

    ## Remove digits
    text = ''.join([i for i in text if not i.isdigit()])

    ## remove mutliple space
    text = re.sub(' +', ' ', text)

    return text


In [4]:
from scipy.spatial.distance import cosine
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer=TfidfVectorizer()

In [5]:
def distance(f1,f2):
    return cosine(
        tfidf_vectorizer.transform([f1]).toarray(),
        tfidf_vectorizer.transform([f2]).toarray(),
        )

In [6]:
def Entites(requet):
    doc = nlp(requet)
    sentences = [x for x in doc.sents]
    print(sentences)
    a=[(X.text.replace(' ','_')) for X in doc.ents]
    entitie=[(x.lemma_[0].upper()+x.lemma_[1:]) for x in [y 
                                      for y
                                      in nlp(str(sentences)) 
                                      if not y.is_stop and (y.pos_ == 'NOUN' or y.pos_ == 'PROPN' )]#or y.pos_ == 'PROPN'
                                      ]
    entities=a
    for element in entitie:
        if element not in entities:
            entities.append(element)
    return entities

In [23]:

from rdflib import Graph
from SPARQLWrapper import SPARQLWrapper, JSON, N3
lien1=[]
entities2=[]
sparql = SPARQLWrapper('https://dbpedia.org/sparql')
def Requete(requetS):
    global lien1
    lien1=[]
    data=[utils_preprocess_text(requetS, flg_stemm=False, flg_lemm=True)]
    global resultat
    global entities2
    entities2=Entites(requetS)
    #print(entities2)
    seuilf=0.0
    for entite in entities2:
        
        seuil=0.0
        sparql.setQuery('''
             PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
             PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
             PREFIX rd: <http://dbpedia.org/resource/>
             PREFIX dbo:<http://dbpedia.org/ontology/>
             SELECT DISTINCT ?object $abs ?comment 
             WHERE { rd:'''+ entite +'''  dbo:wikiPageWikiLink $object .
             $object rdfs:comment $comment.
             $object dbo:abstract $abs.
             FILTER(lang(?abs)='en')
             FILTER(lang(?comment)='en')
            }
        ''')
        #print(entite)
        sparql.setReturnFormat(JSON)
        qres = sparql.query().convert()
       
        #print(qres)
        for result in qres['results']['bindings']:
            data.append(utils_preprocess_text(result['comment']['value'], flg_stemm=False, flg_lemm=True))
            
             #print(result['object']['value']," \nAbstract is :",result['comment']['value'])
        tfidf_vector=tfidf_vectorizer.fit_transform(data)
        
        for result in qres['results']['bindings']:
            dist=1-distance(utils_preprocess_text(requetS, flg_stemm=False, flg_lemm=True) ,utils_preprocess_text(result['comment']['value'], flg_stemm=False, flg_lemm=True))
            # print(wv.wmdistance(requet ,result['comment']['value']))
            if dist >seuil :
                seuil=dist
                #print(seuil)
                resultat=result
        #print(entite,resultat['object']['value'])
        if not qres['results']['bindings'] :
            lien1.append("this entity does not exist in dbpedia")
        else:
            lien1.append(resultat['object']['value'])
        #lang, value = result['object']['xml:lang'], result['object']['value']
    #print(f'Lang: {vars}\tValue: {subject}')
    # if lang == 'en':
        # print(value)
#print(requet,resultatf['object']['value'])


In [24]:
import webbrowser
def recherche():
    req=entree.get()
    doc = nlp(req[0].upper()+req[1:])
    sentences = [x for x in doc.sents]
    entity=[]
    liens=[]
    for sent in sentences:
        Requete(str(sent))
        entity.extend(entities2)
        #print(entity)
        liens.extend(lien1)
    global canvas
    canvas.destroy()
    canvas = Canvas(fenetre, width=550, height=320)
    canvas.pack() 
    link={}
    for i in range(len(entity)):
        b=Label(canvas , text=entity[i])
        b.grid(row =i, column =0,padx=5, pady=5)
        if i==0:
            link = Label(canvas, text=liens[i], fg="#116562")#cursor="hand2"
            link.grid(row =i, column =1,padx=5, pady=5)
            link.bind("<Button-1>", lambda event: webbrowser.open(link.cget("text")))       
        elif i==1:
            link1 = Label(canvas, text=liens[i], fg="#116562", cursor="hand2")
            link1.grid(row =i, column =1,padx=5, pady=5)
            link1.bind("<Button-1>", lambda event: webbrowser.open(link1.cget("text")))
        elif i==2:
            link2 = Label(canvas, text=liens[i], fg="#116562", cursor="hand2")
            link2.grid(row =i, column =1,padx=5, pady=5)
            link2.bind("<Button-1>", lambda event: webbrowser.open(link2.cget("text")))
        elif i==3:
            link3 = Label(canvas, text=liens[i], fg="#116562", cursor="hand2")
            link3.grid(row =i, column =1,padx=5, pady=5)
            link3.bind("<Button-1>", lambda event: webbrowser.open(link3.cget("text")))
        elif i==4:
            link4 = Label(canvas, text=liens[i], fg="#116562", cursor="hand2")
            link4.grid(row =i, column =1,padx=5, pady=5)
            link4.bind("<Button-1>", lambda event: webbrowser.open(link4.cget("text")))
        elif i==5:
            link5 = Label(canvas, text=liens[i], fg="#116562", cursor="hand2")
            link5.grid(row =i, column =1,padx=5, pady=5)
            link5.bind("<Button-1>", lambda event: webbrowser.open(link5.cget("text")))
        elif i==6:
            link6 = Label(canvas, text=liens[i], fg="#116562", cursor="hand2")
            link6.grid(row =i, column =1,padx=5, pady=5)
            link6.bind("<Button-1>", lambda event: webbrowser.open(link6.cget("text")))
        elif i==7:
            link7 = Label(canvas, text=liens[i], fg="#116562", cursor="hand2")
            link7.grid(row =i, column =1,padx=5, pady=5)
            link7.bind("<Button-1>", lambda event: webbrowser.open(link7.cget("text")))
        elif i==8:
            link8 = Label(canvas, text=liens[i], fg="#116562", cursor="hand2")
            link8.grid(row =i, column =1,padx=5, pady=5)
            link8.bind("<Button-1>", lambda event: webbrowser.open(link8.cget("text")))
        else :
            link9 = Label(canvas, text=liens[i], fg="#116562", cursor="hand2")
            link9.grid(row =i, column =1,padx=5, pady=5)
            link9.bind("<Button-1>", lambda event: webbrowser.open(link9.cget("text")))
#from PyQt4 import QtGui
from tkinter import * 
fenetre = Tk() # ouvre une instance de fenêtre
fenetre.geometry('900x600')
fenetre.title("Web Sementique ~ Moteur de recherche")

#set window color
#fenetre.configure(bg ='cadetblue', bd =2, relief=SUNKEN)
fenetre.config(bg='#116562')

# set minimum window size value
fenetre.minsize(900, 600)
 
# set maximum window size value
fenetre.maxsize(900, 600)

"""
#Placer l'interface au Centre

def center(toplevel):
    toplevel.update_idletasks()

    # Tkinter way to find the screen resolution
    # screen_width = toplevel.winfo_screenwidth()
    # screen_height = toplevel.winfo_screenheight()

    # PyQt way to find the screen resolution
    app = QtGui.QApplication([])
    screen_width = app.desktop().screenGeometry().width()
    screen_height = app.desktop().screenGeometry().height()

    size = tuple(int(_) for _ in toplevel.geometry().split('+')[0].split('x'))
    x = screen_width/2 - size[0]/2
    y = screen_height/2 - size[1]/2

    toplevel.geometry("+%d+%d" % (x, y))
    toplevel.title("Centered!")
    
win = Tk.Toplevel(fenetre)
center(win)
"""

label = Label(fenetre, text="Entity Disambiguation ",font="Arial 16 italic",bg="#116562") # fg="red"
label.pack(side=TOP,padx=30, pady=30)
value = StringVar() # DoubleVar 0.0 , IntVar 0, StringVar ‘’
value.set("")
entree = Entry(fenetre, textvariable=value,font="Arial 16 italic", width=150) 
entree.pack(padx=60, pady=25)
entree.insert(0,"Please insert your sentence here...")
entree.pack(padx=25, pady=25)
entree.configure(state=DISABLED)
def on_click(event):
    entree.configure(state=NORMAL)
    entree.delete(0, END)
    # make the callback only work once
    entree.unbind('<Button-1>', on_click_id)

on_click_id = entree.bind('<Button-1>', on_click)
from tkinter import font
myFont = font.Font(size=15)
b1=Button(fenetre , text="Search",bg='cadetblue', fg='#ffffff',font="Arial 16 italic", command=recherche)
b1['font'] = myFont

b1.pack(side=TOP, padx=7, pady=7)
canvas = Canvas(fenetre, bg='white',width=550, height=320 )
canvas.pack() 

fenetre.mainloop()

[Sebastian Thrun works at Google]


# Sebastian Thrun works at Google
# Oracle Database
# luxury vehicle brand of Jaguar Land Rover
# Cristiano Ronaldo
# Messi
# Mac OS X Jaguar is a good operating System 